<a href="https://colab.research.google.com/github/dakilaledesma/Atomized_NN_Experiments/blob/main/benchmarks/CNXT_448_Original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 120 ms, sys: 28.4 ms, total: 148 ms
Wall time: 19.4 s


In [2]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10691, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 10691 (delta 109), reused 163 (delta 87), pack-reused 10469
Receiving objects: 100% (10691/10691), 20.38 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (7819/7819), done.


In [3]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model convnext_base_384_in22ft1k --opt adabelief --warmup-lr 1e-4 --lr 1e-3 --weight-decay 1e-2 --opt-eps 1e-8 --epochs 36 --decay-epochs 3 --cooldown-epochs 0  --sched cosine -b 64 --input-size 3 448 448 --num-classes=300 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth)
Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_22k_1k_384.pth
Model convnext_base_384_in22ft1k created, param count:87873964
Data processing configuration for current model + dataset:
	input_size: (3, 448, 448)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225)
